In [1]:
# MIT License
#
#@title Copyright (c) 2021 CCAI Community Authors { display-mode: "form" }
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

#ClimateLearn: Machine Learning for  Predicting Weather and Climate Extremes


Author(s):
*   Hritik Bansal, University of California Los Angeles, hbansal@g.ucla.edu
*   Shashank Goel, University of California Los Angeles, shashankgoel@g.ucla.edu
*   Tung Nguyen,   University of California Los Angeles, tungnd@g.ucla.edu
*   Aditya Grover, University of California Los Angeles, adityag@cs.ucla.edu

# Table of Contents

*   [Target Audience](#target-audience)
*   [Overview](#overview)
*   [Contributions](#contributions)
*   [Software Requirements](software-requirements)
*   [Data Source](data-source)
*   [Temporal Forecasting](temporal-forecasting)
*   [Spatial Downscaling](spatial-downscaling)

<a name="target-audience"></a>
# Target Audience


*   Climate scientists seeking to explore ML tools, technologies, and resources to tackle a domain-specific problem in climate change. 
*   Data scientists with prior background in deep learning looking for concrete examples on how to model climate extremes using AI.

<a name="overview"></a>
# Overview

Climate change has led to a rapid increase in the occurrence of extreme weather events globally, including floods, droughts, and wildfires. In the longer term, some regions will experience aridification while others will risk sinking due to rising sea levels. Typically, such predictions are done via weather and climate models that simulate the physical interactions between the atmospheric, oceanic, and land surface processes that operate at different scales. Due to the inherent complexity, these climate models can be inaccurate or computationally expensive to run, especially for detecting climate extremes at high spatiotemporal resolutions. In this tutorial, we aim to introduce the participants to machine learning approaches for addressing two fundamental challenges:

(1) **Temporal Forecasting**: The goal is to predict climate variables into the future. \
(2) **Spatial Downscaling**: The goal is to learn a mapping that transforms low-resolution outputs of climate models into high-resolution regional forecasts. 

We illustrate the two key challenges in predicting temperature, a key indicator of global warming levels around the world, in the figure below. Given a temperature grid at a particular resolution **R** at a time stamp **T**, temporal forecasting focuses on predicting the temperature at a future time stamp **T+H** where **H** is the prediction horizon whereas the climate downscaling focuses on predicting the temperature at the same timestep T but at a higher resolution **(R' > R)**. 

<a name="contributions"></a>
# Contributions

Through introductory presentations and colab notebooks, we aim to expose the participants to: 

(a) **APIs** for accessing and navigating popular repositories that host global climate data, such as the Copernicus data store. \
(b) Identifying **relevant datasets** and providing functionality to preprocess them. \
(c) **Algorithms** for training machine learning models. \
(d) **Metrics** for evaluating model performance. \
(e) **Visualization** tools for both the dataset and predicted outputs. 

<a name="software-requirements"></a>
# Software Requirements
This notebook requires the following libraries:
*   climate_learn (pip)

`climate_learn` contains the source files used for modeling climate extremes.

The package is written using `PyTorch` machine learning library.

In [4]:
%%capture
!pip install git+https://github.com/tung-nd/climate_learn.git

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')
# !pwd

import os

# Replace <path_to_directory> with the path to the directory where your files are stored.
os.chdir('2m_temperature')

# Test that the directory has been changed to the correct one.
print(os.getcwd())

/Users/linustws/Desktop/climate-predict/2m_temperature


<a name="data-source"></a>
# Data Source

[1] provides a comprehensive list of typical datasets and benchmarks for weather prediction. 

Among the list of popular datasets, we focus this tutorial on the **ERA5** dataset that provides an hourly estimate of a large number of atmospheric, land and oceanic climate variables. 

<br>
<center><img src="https://cds.climate.copernicus.eu/sites/default/files/cdsapp/copernicus-logo.png" width=300></center>
<br>

ERA5 consists of the climate variable data from 1959 to present. The ERA5 dataset is freely-accessible to the community on the Climate Data Store [(CDS)](https://cds.climate.copernicus.eu/#!/search?text=ERA5&type=dataset) hosted by the European Union's Earth observation programme, [Copernicus](https://www.copernicus.eu/en/about-copernicus).  The users can visit [https://cds.climate.copernicus.eu/api-how-to](https://cds.climate.copernicus.eu/api-how-to#install-the-cds-api-key) to install the CDS API key required to access their data. 




The authors from [2] (WeatherBench dataset) downloaded the ERA5 data from CDS API for a subset of climate variable and preprocessed it at multiple resolutions. The data is freely accessible to anyone at [https://dataserv.ub.tum.de/index.php/s/m1524895?path=%2F](https://dataserv.ub.tum.de/index.php/s/m1524895?path=%2F).

<br>
<center><img src="https://github.com/ai4environment/WeatherBench/raw/master/figures/logo_text_left.png?raw=true" width=500></center>
<br>

_Our package provides the users with a way to download data from ERA5 as well as WeatherBench depending upon their application._



<br/><br/>

**References:**

[1] Ren X, Li X, Ren K, Song J, Xu Z, Deng K, Wang X. Deep learning-based weather prediction: a survey. Big Data Research. 2021 Feb 15;23:100178 [(Paper)](https://www.sciencedirect.com/science/article/pii/S2214579620300460). \
[2] Rasp S, Dueben PD, Scher S, Weyn JA, Mouatadid S, Thuerey N. WeatherBench: a benchmark data set for data‐driven weather forecasting. Journal of Advances in Modeling Earth Systems. 2020 Nov;12(11):e2020MS002203 [(Paper)](https://arxiv.org/abs/2002.00469).

<a name="temporal-forecasting"></a>
# Temporal Forecasting


A precise and reliable weather forecasting is of great importance in various aspect of society including precipitation forecasts are essential for agricultural needs, wind speed and solar power forecasts for energy generation.

<br>
<center><img src="https://drive.google.com/uc?export=view&id=1_tsaaogqzkYVV0jdawnO9GCJTToW_FCi" height=300></center>

The forecasting task can be categorized into (a) **nowcasting** (timescale of a few hours), (b) weather-scale prediction (typically 1day - 1week), (c) **seasonal** prediction (typically months) and (d) **multi-year or decadal** (timescale of multiple years).

In this tutorial, we shall focus on the **medium-range** weather-scale  prediction of the climate variables i.e., typically 3-5 days in the future. This colab notebook demonstrates the temporal forecasting of the *Temperature* variable at *2m* height above the earth's surface. This variable serves as a good indicator of future temperatures on the Earth's surface for the forecasters. 

We shall further use the 2m temperature data at 5.625 degree resolution that divides the Earth's surface into a latitude x longitude grid of 32 x 64.

<br/><br/>



**References:**
1. Rasp S, Dueben PD, Scher S, Weyn JA, Mouatadid S, Thuerey N. WeatherBench: a benchmark data set for data‐driven weather forecasting. Journal of Advances in Modeling Earth Systems. 2020 Nov;12(11):e2020MS002203 [(Paper)](https://arxiv.org/abs/2002.00469).
2. Civitarese DS, Szwarcman D, Zadrozny B, Watson C. Extreme Precipitation Seasonal Forecast Using a Transformer Neural Network. arXiv preprint arXiv:2107.06846. 2021 Jul 14. [(Paper)](https://arxiv.org/abs/2107.06846)
3. Sønderby CK, Espeholt L, Heek J, Dehghani M, Oliver A, Salimans T, Agrawal S, Hickey J, Kalchbrenner N. Metnet: A neural weather model for precipitation forecasting. arXiv preprint arXiv:2003.12140. 2020 Mar 24. [(Paper)](https://arxiv.org/pdf/2003.12140.pdf)


In [7]:
from climate_learn.data import download

In [8]:
# Download data from copernicus (~15-20 mins)
# Generate API KEY: https://cds.climate.copernicus.eu/api-how-to
# api_key = "154140:40d3d2e0-ed2c-4f60-8bc8-d15789841be0" # Change to your_api_key
# download(source = "copernicus", variable = "2m_temperature", dataset = "era5", year = 1979, api_key = api_key)

In [ ]:
# Download data from weatherbench (~2-3 minutes)
download(root = "/content/drive/MyDrive/Climate/.climate_tutorial", source = "weatherbench", variable = "2m_temperature", dataset = "era5", resolution = "5.625")

ERA5 dataset directory structure from the Weatherbench source.

```
|-- 5.625deg
|   |-- 2m_temperature
|       |-- 2m_temperature_1979_5.625deg.nc
|       |-- 2m_temperature_1980_5.625deg.nc
|       |-- ...
|       |-- 2m_temperature_2018_5.625deg.nc
```

## Data Preprocessing


The data is stored in the [NetCDF](https://en.wikipedia.org/wiki/NetCDF) files with _.nc_ extension. One of the distinct features of this format is the **named** specification to the coordinates and the data variables. 

As shown below, we first merge all the yearly NetCDF files, and display the structure of the format. xarray library is used to read the NetCDF files. It allows the users to manipulate data based on more informative labels instead of integer location. 



In [9]:
from climate_learn.utils.data import load_dataset, view

dataset = load_dataset(os.getcwd())
view(dataset)

<xarray.Dataset>
Dimensions:  (lon: 64, lat: 32, time: 254256)
Coordinates:
  * lon      (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
  * lat      (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
  * time     (time) datetime64[ns] 1979-01-01 ... 2018-12-31T23:00:00
Data variables:
    t2m      (time, lat, lon) float32 dask.array<chunksize=(8760, 32, 64), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2019-11-07 07:51:20 GMT by grib_to_netcdf-2.14.0: /opt/ecmw...

## Data Conversion
We further convert the *NetCDF* files to *PyTorch* Dataloaders.

**Pros**: We can use the dataloaders for training and evaluating neural networks.\
**Cons**: We loose useful meta information (such as 'time', 'location') during conversion as dataloaders only allow for integer location based treatment. 

We store the useful information about
 the data ('lat', 'long') of the regions as _data members_ of our dataloaders.





### Example

- In the given example, we train our model from the data from 1979 (start train year) -2015, validate it from the data from 2015 (start validation year) - 2016, and test it on data from 2017 (start test year) - 2018 (end test year).
- We split the data this way as it allows us to test the temporal generalization of our models. Additionally, it replicates more practical setting where the model that is trained on the historical data is used for forecasting in the future.
- pred_range = Days(3) indicates that we are going to predict 3 days in the future.
- subsample = Hours(6) indicates that we shall use the data at every 6 hours from 24 hours of the data from a single day i.e., rather than using data at 12am, 1am, 2am...11pm from a single day, we shall use the data at 12am, 6am, 12pm, 6pm. As climate variables are highly correlated within a window of a few hours, subsampling helps in removing those redundancies during training.


In [10]:
from climate_learn.utils.datetime import Year, Days, Hours
from climate_learn.data import DataModule

data_module = DataModule(
    dataset = "ERA5",
    task = "forecasting",
    root_dir = "/content/drive/MyDrive/Climate/.climate_tutorial/data/weatherbench/era5/5.625/",
    in_vars = ["2m_temperature"],
    out_vars = ["2m_temperature"],
    train_start_year = Year(1979),
    val_start_year = Year(2015),
    test_start_year = Year(2017),
    end_year = Year(2018),
    pred_range = Days(3),
    subsample = Hours(6),
    batch_size = 128,
    num_workers = 1
)

TypeError: __init__() got an unexpected keyword argument 'dataset'

## Neural Networks Architectures

We consider three deep neural network architectures for in this tutorial.

1. Convolutional Neural Networks (CNN)
<center><img src="https://viso.ai/wp-content/uploads/2021/03/cnn-convolutional-neural-networks-1060x362.jpg" width=400></center>

CNN is a widely used neural network architecture for visual recognition. A CNN is a constrainted version of a regular neural network that takes advantage of the prior knowledge that the input is an image. The architecture can be flexibly adapted to various image resolutions. 

For our use case, the latitude-longitude gridded data is treated as an image composed of climate variables in its channel instead of the Red-Green-Blue (RGB) information.

Resource(s): https://cs231n.github.io/convolutional-networks/ 

Among the long list of CNN variants availablely, we provide support for its popular variants. 


Variants of CNN architecture: \

a. **ResNet**

<center><img src="https://miro.medium.com/max/875/1*WpX_8eCeTsEcCs8vdXtUCw.png" width=400></center>

ResNets have been used to achieve SOTA weather forecasting using neural networks for temperature and geopotential in [1]. 

Paper: [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)

b. **U-Net**

<center><img src="https://miro.medium.com/max/875/1*f7YOaE4TWubwaFF7Z1fzNw.png" width=400></center>

The basic building blocks of the U-Net architecture involve downsampling as well as upsampling convolutions. The downsampling blocks project the input from higher dimension to a lower dimension, and upsampling blocks project the low dimension latent space to the higher dimension input space. After gaining popularity in the Biomedical domain, our package allows the users to benchmark U-Net in the Climate modeling space too.


Paper: [U-Net: Convolutional Networks for Biomedical Image Segmentation
](https://arxiv.org/abs/1505.04597) 



2. Vision Transformers

Vision transformers are the latest contemporary to CNN variants for visual recognition. We relegate the audience to the related paper for its architectural details.

<center><img src="https://viso.ai/wp-content/uploads/2021/09/vision-transformer-vit.png" width=400></center>

Vision Transformers have gained immense popularity in the Vision community, and its usefulness to learn representations of climate variables is still under-explored. [2] used Transformers for short-range temperature forecasting.
We believe that our ViT implementation shall allow the users to benchmark ViT on climate modeling tasks.

Paper: <a href="https://arxiv.org/abs/2010.11929">An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale</a>

<br/><br/>


**References**:

[1] Rasp S, Thuerey N. Data‐driven medium‐range weather prediction with a resnet pretrained on climate simulations: A new model for weatherbench. Journal of Advances in Modeling Earth Systems. 2021 Feb;13(2):e2020MS002405.\
[2] Bilgin O, Mąka P, Vergutz T, Mehrkanoon S. TENT: Tensorized encoder transformer for temperature forecasting. arXiv preprint arXiv:2106.14742. 2021 Jun 28.



<br/><br/>

In this tutorial, we shall demonstrate the training of a resnet from scratch. It is important to note that the choice of model architecture and hyperparameters are for demonstration purposes only. 

## Model initialization 

The hyperparameters and ResNet architecture chosen allow for a model that forecasts with 85.7% test accuracy, while still training within a reasonable amount of time for the sake of the tutorial (by nature of being a smaller model). We leave it to the user to perform a more exhaustive search of hyperparameter values for training models that perform better.

In [ ]:
from climate_learn.models import load_model

# model_kwargs = {
#     "img_size": [32, 64],
#     "patch_size": 2,
#     "drop_path": 0.1,
#     "drop_rate": 0.1,
#     "learn_pos_emb": True,
#     "in_vars": data_module.hparams.in_vars,
#     "out_vars": data_module.hparams.out_vars,
#     "embed_dim": 128,
#     "depth": 8,
#     "decoder_depth": 0,
#     "num_heads": 4,
#     "mlp_ratio": 4,
# }

model_kwargs = {
    "in_channels": len(data_module.hparams.in_vars),
    "out_channels": len(data_module.hparams.out_vars),
    "n_blocks": 4
}

optim_kwargs = {
    "lr": 1e-4,
    "weight_decay": 1e-5,
    "warmup_epochs": 1,
    "max_epochs": 5,
}

# model_module = load_model(name = "vit", task = "forecasting", model_kwargs = model_kwargs, optim_kwargs = optim_kwargs)
model_module = load_model(name = "resnet", task = "forecasting", model_kwargs = model_kwargs, optim_kwargs = optim_kwargs)
# model_module = load_model(name = "unet", task = "forecasting", model_kwargs = model_kwargs, optim_kwargs = optim_kwargs)

In [ ]:
from climate_learn.models import set_climatology
set_climatology(model_module, data_module)

## Training


The training objective ensures that the machine learning model makes accurate forecasts over the gridded data. We employ latitude weighted RMSE given by:

<br>
$RMSE = \frac{1}{N_{forecasts}}\sum_{i}^{N_{forecasts}}\sqrt{\frac{1}{N_{lat}N_{lon}}\sum_{j}^{N_{lat}}\sum_{k}^{N_{lon}}L(j)(f_{i,j,k}-t_{i,j,k})^{2}} \tag{1}$ 
<br>

where $f$ is the model forecast and $t$ is the ERA5 truth. $L(j)$ is the latitude weighing factor at the $j^{th}$ latitude index:

<br>
$L(j) = \frac{cos(lat(j))}{\frac{1}{N_{lat}}\sum_{j}^{N_{lat}}cos(lat(j))} \tag{2}$
<br>

(Optional) If you want to monitor training and validation curves of the model using [Weights and Biases](https://docs.wandb.ai/), uncomment the lines in the following code block and login to your Wandb account (only once).

In [ ]:
from climate_learn.training import Trainer, WandbLogger

trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 5,
    # logger = WandbLogger(project = "climate_tutorial", name = "forecast-vit")
)

In [ ]:
trainer.fit(model_module, data_module)

## Evaluation 


Once our prediction model is trained, we want to be able to evaluate it against the ground truth labels for data samples in the test set. 

In addition to the Latitude weighted RMSE (Eq. 1), we shall look at the Anomaly Correlation Coefficient (ACC) which is defined as:

<br>
$ACC = \frac{\sum_{i,j,k}L(j)f'_{i,j,k}t'_{i,j,k}}{\sqrt{\sum_{i,j,k}L(j)f'^{2}_{i,j,k}L(j)t'^{2}_{i,j,k}}} \tag{3}$
<br>

where $'$ denotes the difference to the climatology. We define climatology as:

<br>
$climatology_{j,k} = \frac{1}{N_{time}}\sum{t_{j,k}}\tag{4}$
<br>

For the RMSE metric, we compare the deep learning model with a climatological forecast.

In [ ]:
trainer.test(model_module, data_module)

The model's prediction has a strong correlation with the ground truth, which is indicated by a high ACC value. Compared to a climatological forecast, the deep learning model achieves a much smaller RMSE error.

## Visualization 

We visualize the **bias**, given by the difference in the predicted and the ground truth values, to better analyze our learned model.

Visualization is done on the test set. We can either specify exact time for the initial condition, or randomly sample from the test set.

In [ ]:
# from climate_tutorial.utils import visualize

import os
import random
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

def visualize(model_module, data_module, split = "test", samples = 2, save_dir = None):
    if save_dir is not None:
        os.makedirs(save_dir, exist_ok = True)

    # dataset.setup()
    dataset = eval(f"data_module.{split}_dataset")

    if(type(samples) == int):
        idxs = random.sample(range(0, len(dataset)), samples)
    elif(type(samples) == list):
        idxs = [np.searchsorted(dataset.time, np.datetime64(datetime.strptime(dt, "%Y-%m-%d:%H"))) for dt in samples]
    else:
        raise Exception("Invalid type for samples; Allowed int or list[datetime.datetime or np.datetime64]")

    fig, axes = plt.subplots(len(idxs), 4, figsize=(30, 3 * len(idxs)), squeeze = False)

    for index, idx in enumerate(idxs):
        x, y, _, _ = dataset[idx] # 1, 1, 32, 64
        pred = model_module.forward(x.unsqueeze(0)) # 1, 1, 32, 64

        inv_normalize = model_module.denormalization
        init_condition, gt = inv_normalize(x), inv_normalize(y)
        pred = inv_normalize(pred)
        bias = pred - gt

        for i, tensor in enumerate([init_condition, gt, pred, bias]):
            ax = axes[index][i]
            im = ax.imshow(tensor.detach().squeeze().cpu().numpy())
            im.set_cmap(cmap=plt.cm.RdBu)
            fig.colorbar(im, ax=ax)

        if(data_module.hparams.task == "forecasting"):
            axes[index][0].set_title("Initial condition [Kelvin]")
            axes[index][1].set_title("Ground truth [Kelvin]")
            axes[index][2].set_title("Prediction [Kelvin]")
            axes[index][3].set_title("Bias [Kelvin]")
        elif(data_module.hparams.task == "downscaling"):
            axes[index][0].set_title("Low resolution data [Kelvin]")
            axes[index][1].set_title("High resolution data [Kelvin]")
            axes[index][2].set_title("Downscaled [Kelvin]")
            axes[index][3].set_title("Bias [Kelvin]")
        else:
            raise NotImplementedError

    fig.tight_layout()
    
    if save_dir is not None:
        plt.savefig(os.path.join(save_dir, 'visualize.png'))
    else:
        plt.show()

# if samples = 2, we randomly pick 2 initial conditions in the test set
visualize(model_module, data_module, samples = ["2017-06-01:12", "2017-08-01:18"])

The visualization shows that the model makes reasonably accurate predictions in most parts of the globe, and the prediction well correlates with the ground truth. The model seems to make large errors near the two poles, where the temperature is more unpredictable. We can improve the accuracy of the model by either including more input variables, using a more expressive model, or training for longer. For the sake of this tutorial, we use a small model and only train for a short time.

In addition to visualizing the bias the model makes for each individual data point, we can also visualize the mean bias across the entire test set.

In [ ]:
import torch
from tqdm import tqdm

def visualize_mean_bias(model_module, data_module, save_dir = None):
    if save_dir is not None:
        os.makedirs(save_dir, exist_ok = True)

    loader = data_module.test_dataloader()

    all_mean_bias = []
    for batch in tqdm(loader):
      x, y, _, _ = batch # B, 1, 32, 64
      x = x.to(model_module.device)
      y = y.to(model_module.device)
      pred = model_module.forward(x) # B, 1, 32, 64

      inv_normalize = model_module.denormalization
      init_condition, gt = inv_normalize(x), inv_normalize(y)
      pred = inv_normalize(pred)
      bias = pred - gt # B, 1, 32, 64
      mean_bias = bias.mean(dim=0)
      all_mean_bias.append(mean_bias)
    
    all_mean_bias = torch.stack(all_mean_bias, dim=0)
    mean_bias = torch.mean(all_mean_bias, dim=0)

    fig, axes = plt.subplots(1, 1, figsize=(12, 4), squeeze = False)
    ax = axes[0,0]

    im = ax.imshow(mean_bias.detach().squeeze().cpu().numpy())
    im.set_cmap(cmap=plt.cm.RdBu)
    fig.colorbar(im, ax=ax)
    ax.set_title("Mean bias [Kelvin]")

    fig.tight_layout()
    
    if save_dir is not None:
        plt.savefig(os.path.join(save_dir, 'visualize_mean_bias.png'))
    else:
        plt.show()
  
visualize_mean_bias(model_module.cuda(), data_module)

In [ ]:
# # from climate_tutorial.utils import visualize

# import os
# import random
# import numpy as np
# from datetime import datetime
# from plotly.express import imshow
# from plotly.subplots import make_subplots
# import plotly.graph_objects as go

# # more use cases for visualize, make a more intuitive api
# # which split of the data? train/val/test? currently test with a random data point
# # timestamp that we are visualizing?
# # only timestamp works -- can infer the split, we don't have the timestamp info for now -> include it in the dataloader
# # number: 5 data points

# # add lat long information 
# # plotly to zoom in

# samples = ["2017-01-01:12", "2017-02-01:18"]
# def visualize(model_module, data_module, split = "test", samples = samples):
#     # dataset.setup()
#     dataset = eval(f"data_module.{split}_dataset")

#     if(type(samples) == int):
#         idxs = random.sample(range(0, len(dataset)), samples)
#     elif(type(samples) == list):
#         idxs = [np.searchsorted(dataset.time, np.datetime64(datetime.strptime(dt, "%Y-%m-%d:%H"))) for dt in samples]
#     else:
#         raise Exception("Invalid type for samples; Allowed int or list[datetime.datetime or np.datetime64]")

#     # print(dataset.time[idxs[0]])
#     # row_titles = [datetime.strftime(None, "%Y-%m-%d:%H") for idx in idxs]

#     if(data_module.hparams.task == "forecasting"):
#         col_titles = ["Initial condition", "Ground truth", "Prediction", "Bias"]
#     elif(data_module.hparams.task == "downscaling"):
#         col_titles = ["Low resolution data", "High resolution data", "Downscaled", "Bias"]
#     else:
#         raise NotImplementedError

#     fig = make_subplots(len(idxs), 4, subplot_titles = col_titles * len(idxs))
#     for i, idx in enumerate(idxs):
#         x, y, _, _ = dataset[idx] # 1, 1, 32, 64
#         pred = model_module.forward(x.unsqueeze(0)) # 1, 1, 32, 64

#         inv_normalize = model_module.denormalization
#         init_condition, gt = inv_normalize(x), inv_normalize(y)
#         pred = inv_normalize(pred)
#         bias = pred - gt

#         for j, tensor in enumerate([init_condition, gt, pred, bias]):
#             fig.add_trace(imshow(tensor.detach().squeeze().cpu().numpy(), color_continuous_scale = "rdbu", x = dataset.inp_lon if i == 0 else dataset.out_lon, y = dataset.inp_lat if i == 0 else dataset.out_lat).data[0], row = i + 1, col = j + 1)
#             # fig.colorbar(im, ax=ax)

#     # fig.tight_layout()
#     fig.show()

# visualize(model_module, data_module)

<a name="spatial-downscaling"></a>
# Spatial Downscaling


General Circulation Models (GCMs) provide us with the future projections of climate scenarios. These raw estimates have to be downscaled at the desired resolution for actionable guidance.

<br>
<center><img src="https://drive.google.com/uc?export=view&id=11i2CIRxlVRqOHIgZRABwF05Qf5KeqVwc" height=300></center>

In practice, statistical spatial downscaling can be used to make predictions about a climate variable (a) over the latitude-longitude grid of **higher** resolution than the input grid and (b) on specific sites at the target locations. For example, we can predict the temperature at a specific station in Germany based on the gridded temperature data over the whole country.

Major class of statistical downscaling models include Perfect Prognosis (PP) [1] that aims at learning a transfer function $$\hat{y} = f(x, Z)$$ where $y$ is the true value at location $x$ and $Z$ are the set of model predictors for the climate model. The various PP models differ in their realization of the transfer function $f$. Related works in [2] provides deeper details into the previous works. In [2,3], the authors use CNNs as the transfer function, broadly due to its inherent inductive bias towards handling Vision data. The ability of Deep CNNs to perform super-resolution is a well-explored field of study [4].

<br/><br/>

**References:**
1. Maraun D, Wetterhall F, Ireson AM, Chandler RE, Kendon EJ, Widmann M, Brienen S, Rust HW, Sauter T, Themeßl M, Venema VK. Precipitation downscaling under climate change: Recent developments to bridge the gap between dynamical models and the end user. Reviews of geophysics. 2010 Sep;48(3).
2. Vaughan A, Tebbutt W, Hosking JS, Turner RE. Convolutional conditional neural processes for local climate downscaling. arXiv preprint arXiv:2101.07950. 2021 Jan 20.
3. Baño-Medina J, Manzanas R, Gutiérrez JM. Configuration and intercomparison of deep learning neural models for statistical downscaling. Geoscientific Model Development. 2020 Apr 28;13(4):2109-24.
4. Yamanaka J, Kuwashima S, Kurita T. Fast and accurate image super resolution by deep CNN with skip connection and network in network. InInternational Conference on Neural Information Processing 2017 Nov 14 (pp. 217-225). Springer, Cham.

- In this tutorial, we shall focus on mapping the coarse resolution data for a variable to a finer resolution at a given time stamp. Specifically, we shall continue with focusing on the _Temperature at 2m_ climate variable using a ResNet model.

## Data Download

To perfrom climate downscaling, we need to have data for the temperature at 2m at different resolutions. In addition to the 5.625deg dataset we downloaded above, here we download the 2.8125deg dataset, which divides the Earth's surface into a latitude x longitude grid of 64 x 128.

In [ ]:
from climate_learn.data import download

# Download data from weatherbench (~2-3 minutes)
# download(root = "/content/drive/MyDrive/Climate/.climate_tutorial", source = "weatherbench", variable = "2m_temperature", dataset = "era5", resolution = "5.625")
# Download data from weatherbench (~4-6 minutes)
download(root = "/content/drive/MyDrive/Climate/.climate_tutorial", source = "weatherbench", variable = "2m_temperature", dataset = "era5", resolution = "2.8125")

## Data Conversion

In [ ]:
from climate_learn.utils.datetime import Year, Days, Hours
from climate_learn.data import DataModule

data_module = DataModule(
    dataset = "ERA5",
    task = "downscaling",
    root_dir = "/content/drive/MyDrive/Climate/.climate_tutorial/data/weatherbench/era5/5.625",
    root_highres_dir = "/content/drive/MyDrive/Climate/.climate_tutorial/data/weatherbench/era5/2.8125",
    in_vars = ["2m_temperature"],
    out_vars = ["2m_temperature"],
    train_start_year = Year(1979),
    val_start_year = Year(2015),
    test_start_year = Year(2017),
    end_year = Year(2018),
    subsample = Hours(6),
    batch_size = 128,
    num_workers = 1
)

## Model initialization

In [ ]:
from climate_learn.models import load_model

# model_kwargs = {
#     "img_size": [32, 64],
#     "patch_size": 4,
#     "drop_path": 0.1,
#     "drop_rate": 0.1,
#     "learn_pos_emb": True,
#     "in_vars": data_module.hparams.in_vars,
#     "out_vars": data_module.hparams.out_vars,
#     "upsampling": data_module.train_dataset.downscale_ratio,
#     "embed_dim": 128,
#     "depth": 8,
#     "decoder_depth": 0,
#     "num_heads": 4,
#     "mlp_ratio": 4,
# }

model_kwargs = {
    "in_channels": len(data_module.hparams.in_vars),
    "out_channels": len(data_module.hparams.out_vars),
    "upsampling": data_module.train_dataset.downscale_ratio,
    "n_blocks": 4,
}

optim_kwargs = {
    "optimizer": "adamw",
    "lr": 1e-4,
    "weight_decay": 1e-5,
    "warmup_epochs": 1,
    "max_epochs": 5,
}

# model_module = load_model(name = "vit", task = "downscaling", model_kwargs = model_kwargs, optim_kwargs = optim_kwargs)
model_module = load_model(name = "resnet", task = "downscaling", model_kwargs = model_kwargs, optim_kwargs = optim_kwargs)
# model_module = load_model(name = "unet", task = "downscaling", model_kwargs = model_kwargs, optim_kwargs = optim_kwargs)

In [ ]:
# latitude long info, 
from climate_learn.models import set_climatology
set_climatology(model_module, data_module)

## Training

In [ ]:
from climate_learn.training import Trainer, WandbLogger

trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 5,
    # logger = WandbLogger(project = "climate_tutorial", name = "downscale-vit")
)

In [ ]:
trainer.fit(model_module, data_module)

## Evaluation

In [ ]:
trainer.test(model_module, data_module)

## Visualization 

In [ ]:
# from climate_tutorial.utils import visualize
# visualize(model_module, data_module)
visualize(model_module, data_module, samples = ["2017-06-01:12", "2017-08-01:18"])

## Potential Applications of the Tutorial


1. Our package with this tutorial makes it easy for the practitioners to focus on the machine learning model design and evaluate their models against standard benchmarks such as Weatherbench ([Leaderboard](https://github.com/pangeo-data/WeatherBench)) using our evaluation support. 
2. The climate experts can load and train their customized machine learning models with various hyperparameters. For instance, they can study the effect of varying the capacity of machine learning models on its predictions.
3. To best of our knowledge, this is the only package that provides support for temporal forecasting and spatial downscaling under the hood. It helps the forecasting community to benefit from the improvements in modeling downscaling and vice-versa.

## Limitations


1. Due to the noise involved in capturing climate variables, climate modeling is particularly a challenging task. Different datasets such as ERA5, CMIP6 may contain different values for a particular climate variable. 
2. This tutorial does not cover the aspect of uncertainity in the model predictions. We shall add more capabilities to our models in the package for climate modelers to quantify the error in the predicted output.
3. There is a massive amount of climate data available (~10 TBs), our tutorial does not cover aspects of how to deal with the data at that scale which might be important for capturing more sophisticated climate phenomenon.